In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
import django


# Facts:

Data has no missing values

Not all shop_id in train set are used in test set. Test set excludes following shops (not vice versa)
[0, 1, 8, 9, 11, 13, 17, 20, 23, 27, 29, 30, 32, 33, 40, 43, 51, 54]

Not all item in train set are in test set and vice versa

In test set, a fixed set of items (5100) are used for each shop_id, and each item only appears one per each shop

In [8]:
df_items = pd.read_csv('F:/git_repos/data_competition/predict-future-sales/data/items.csv')
df_items.head(70)

item_name  item_id  \
0           ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1   !ABBYY FineReader 12 Professional Edition Full...        1   
2       ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3     ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4         ***КОРОБКА (СТЕКЛО)                       D        4   
5   ***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ  (UNI)         ...        5   
6            ***УДАР ПО ВОРОТАМ (UNI)               D        6   
7          ***УДАР ПО ВОРОТАМ-2 (UNI)               D        7   
8            ***ЧАЙ С МУССОЛИНИ                     D        8   
9       ***ШУГАРЛЭНДСКИЙ ЭКСПРЕСС (UNI)             D        9   
10          *ЗА ГРАНЬЮ СМЕРТИ                       D       10   
11          *ЛИНИЯ СМЕРТИ                           D       11   
12                    *МИХЕЙ И ДЖУМАНДЖИ  Сука любовь       12   
13          *СПАСАЯ ЭМИЛИ                           D       13   
14          *ЧОКНУТЫЙ ПРОФЕССОР /МАГИЯ/             D       14   
15             //АДРЕНАЛИН: ОДИН ПРОТИВ ВСЕХ (Регион)       15   
16                         //МОНГОЛ С.Бодров (Регион)       16   
17                    //НЕ ОСТАВЛЯЮЩИЙ СЛЕДА (Регион)       17   
18          /БОМБА ДЛЯ НЕВЕСТЫ /2DVD/               D       18   
19                          /ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72       19   
20                                 /ОДНАЖДЫ В КИТАЕ-2       20   
21                                    /ПОСЛЕДНИЙ ШАНС       21   
22                               /ПРОКЛЯТЬЕ ЭЛЬ ЧАРРО       22   
23                                  /СЕВЕР И ЮГ /Ч.2/       23   
24                               /СМЕРТЕЛЬНЫЙ РАСКЛАД       24   
25                                        /ТЫ  - ТРУП       25   
26                   /УМНОЖАЮЩИЙ ПЕЧАЛЬ т.2 (сер.3-4)       26   
27                  007 Legends [PS3, русская версия]       27   
28            007 Legends [PС, Jewel, русская версия]       28   
29             007 Legends [Xbox 360, русская версия]       29   
..                                                ...      ...   
40    100 Best classical melodies (mp3-CD) (Digipack)       40   
41     100 Best relax melodies (mp3-CD) (CD-Digipack)       41   
42     100 Best romantic melodies (mp3-CD) (Digipack)       42   
43                                 100 МИЛЛИОНОВ ЕВРО       43   
44  100 лучших мелодий против стресса (mp3-CD) (CD...       44   
45   100 лучших народных песен (mp3-CD) (CD-Digipack)       45   
46   100 лучших произведений Баха (mp3-CD) (Digipack)       46   
47  100 лучших произведений Бетховена (mp3-CD) (Di...       47   
48  100 лучших произведений Вивальди (mp3-CD) (CD-...       48   
49  100 лучших произведений Моцарта (mp3-CD) (Digi...       49   
50  100 лучших произведений Рахманинова (mp3-CD) (...       50   
51  100 лучших произведений классики (mp3-CD) (Dig...       51   
52  100 лучших произведений фортепианной музыки (m...       52   
53  100 лучших русских романсов (mp3-CD) (CD-Digip...       53   
54  100 лучших цыганских романсов (mp3-CD) (CD-Dig...       54   
55      100 любимых маленьких сказок (mp3-CD) (Jewel)       55   
56  100 любимых песен. Александра Пахмутова (подар...       56   
57  100 пудов из мультхитов (подар. уп.) (mp3-CD) ...       57   
58             100 чудес России [PC, Цифровая версия]       58   
59          100 шедевров классики (mp3-CD) (Digipack)       59   
60  1000 английских фраз на все случаи жизни (mp3-...       60   
61         1000 и одна ночь  2CD (mp3-DVD) (DVD-Book)       61   
62                          10000 ЛЕТ ДО Н.Э. WB (BD)       62   
63                                 101 ДАЛМАТИНЕЦ  wd       63   
64                      101 ДАЛМАТИНЕЦ 2 м/ф (регион)       64   
65  101 ДАЛМАТИНЕЦ 2:ПРИКЛЮЧЕНИЯ ПАТЧА В ЛОНДОНЕ (BD)       65   
66                            101 ДАЛМАТИНЕЦ М/Ф (BD)       66   
67                        101 ДАЛМАТИНЕЦ м/ф (регион)       67   
68  101 Далматинец. Интерактивная сказка [PC-CD, J...       68   
69  101 совет начи

In [2]:
def qianbao_compute(num):
    cost = 0.0055*num
#    cost = 0.0038*num
    return num-cost

num1 = qianbao_compute(377*3 + 650*2)
print('qianbao_compute outcome is ', num1)

qianbao_compute outcome is  2417.6295


In [9]:
2935850/34.0

86348.5294117647

In [4]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9395  0.9317  0.9331  0.9304  0.9430  0.9355  0.0049  
MAE (testset)     0.7396  0.7366  0.7358  0.7323  0.7393  0.7367  0.0027  
Fit time          5.16    5.41    5.10    5.11    5.09    5.17    0.12    
Test time         0.14    0.18    0.16    0.16    0.16    0.16    0.01    


{'fit_time': (5.156637668609619,
  5.409975051879883,
  5.103010177612305,
  5.1074113845825195,
  5.086176872253418),
 'test_mae': array([0.73962007, 0.73659206, 0.73580125, 0.73230627, 0.73929039]),
 'test_rmse': array([0.93945414, 0.93170919, 0.93308197, 0.930446  , 0.94303011]),
 'test_time': (0.13900423049926758,
  0.1784822940826416,
  0.16401886940002441,
  0.15799927711486816,
  0.16357111930847168)}